<h1>Agrupación y Segmentación de Vecindarios</h1>

In [1]:
#Librerias
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
#!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.10

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.3.0                |     pyhd8ed1ab_0          68 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         251 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.3.0-pyhd8ed1ab_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            pkgs/main/linux-64::certifi-2022.12.7~ --> conda-forge/noarch::certifi-

In [5]:

from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # librería para graficar mapas 

print('Importadas')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.10

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.2.2               |     pyhd8ed1ab_0         544 KB  conda-forge
    branca-0.6.0               |     pyhd8ed1ab_0          27 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.10            |          2_cp310           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.2   |  py310hff52083_1         1.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following NEW packages

<h2>Se utiliza BeatifulSoup para obtener los datos de la página web que contiene los códigos postales de Toronto.</h2>

In [8]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

<h2>Seleccionar solo los datos de la tabla que contienen los codigos postales de Toronto.</h2>

In [9]:
table = soup.find('table')

In [10]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)

<h2>Se crea tabla de códigos postales luego de importar el contenidos en la tabla.Luego se le da formato a la tabla usando pandas.</h2>

In [11]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")

In [12]:
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0,"Neigh"],axis=1)

In [13]:
postalcode_list.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [14]:
postalcode_list.shape

(103, 3)